In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [2]:
image = cv2.imread('solidWhiteCurve.jpg')
print('This image is:', type(image), 'with dimesions:', image.shape)

This image is: <class 'numpy.ndarray'> with dimesions: (540, 960, 3)


In [3]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)


# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [4]:
vertices = np.array([[(0,540),(430, 330), (510, 330), (960, 540)]], dtype=np.int32)
image = mpimg.imread('solidWhiteRight.jpg')
gray_img = grayscale(image)
gray_gauss = gaussian_blur(gray_img, 3)
canny_img = canny(gray_gauss, 65, 195)
h_lines = hough_lines(canny_img,1,np.pi/180,50,20,25)


In [5]:
print (type(h_lines), np.shape(h_lines))


<class 'numpy.ndarray'> (60, 1, 4)


In [6]:
y_ = h_lines[:,0,3] - h_lines[:,0,1]
x_ = h_lines[:,0,2] - h_lines[:,0,0]
slope = np.divide(y_, x_)
print (slope)

[ 0.62469136  0.65171504  0.19422572  0.19574468  0.65104167 -0.21167883
 -0.16037736 -0.1734104  -0.13793103 -0.02702703 -0.22857143 -0.09090909
  0.625      -0.234375   -0.25806452 -0.68085106 -0.15942029 -0.13333333
  0.0326087   0.          0.12857143  0.18032787 -0.72164948 -0.26785714
 -0.21621622 -0.30120482 -0.27536232 -0.18918919 -0.2173913  -0.13333333
 -0.24576271 -0.13846154  0.11864407 -0.22222222 -0.10416667 -0.72631579
 -0.23404255 -0.68888889 -0.17105263 -0.17857143 -0.12121212 -0.22222222
 -0.04597701 -0.20618557 -0.17647059 -0.10526316 -0.11940299  0.65142857
 -0.1        -0.10344828 -0.17460317 -0.19008264 -0.03030303 -0.12149533
 -0.12318841 -0.13793103        -inf -0.72093023 -0.7         0.02040816]


/home/clairvoyance/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()


In [7]:
slope[slope <= -1E308] = 0
print(len(slope))

60


In [8]:
positive = slope[slope > 0]
avg_p = np.average(positive)
negative = slope[slope < 0]
avg_n = np.average(negative)
print (np.arctan(avg_p)*180/np.pi, np.arctan(avg_n)*180/np.pi)

18.7540935069 -13.3618141633


In [9]:
pos_pts = h_lines[slope > 0]
pos_x = np.append(pos_pts[:,0,0], pos_pts[:,0,2])
pos_y = np.append(pos_pts[:,0,1], pos_pts[:,0,3])
print (np.shape(pos_x), np.shape(pos_y))

neg_pts = h_lines[slope < 0]


(24,) (24,)


In [16]:
import scipy.stats as sp
m, c, r_value, p_value, std_err = sp.linregress(pos_x,y=pos_y)
print (intercept)

151.07589642


In [21]:
y1 = 540
x1 = int((y1-c)/m)

y2 = 350
x2 = int((y2-c)/m)

new_line = np.zeros((2,1,4), dtype = int)
new_line[0,0,:] = np.array([x1, y1, x2, y2])
new_line[1,0,:] = np.array([x1, y1, x2, y2])

print (y2, new_line[0,0,3])

350 350


In [ ]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    y_ = lines[:,0,3] - lines[:,0,1]
    x_ = lines[:,0,2] - lines[:,0,0]
    slope = np.divide(y_, x_)
    
    lef_pts = lines[slope > 0]
    lef_x = np.append(lef_pts[:,0,0], lef_pts[:,0,2])
    lef_y = np.append(lef_pts[:,0,1], lef_pts[:,0,3])
    m_l, c_l, r_l, p_l, se_l = sp.linregress(lef_x,lef_y)
    yl1 = 540
    xl1 = int((y1-c_l)/m_l)
    yl2 = 350
    xl2 = int((y2-c_l)/m_l)

    
    rig_pts = lines[slope < 0]
    rig_x = np.append(rig_pts[:,0,0], rig_pts[:,0,2])
    rig_y = np.append(rig_pts[:,0,1], rig_pts[:,0,3])
    m_r, c_r, r_r, p_r, se_r = sp.linregress(rig_x,rig_y)
    yr1 = 540
    xr1 = int((y1-c_r)/m_r)
    yr2 = 350
    xr2 = int((y2-c_r)/m_r)

    new_lines = np.zeros((2,1,4), dtype = int)
    new_lines[0,0,:] = np.array([xl1, yl1, xl2, yl2])
    new_lines[1,0,:] = np.array([xr1, yr1, xr2, yr2])
    
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, new_lines)
    
